In [1]:
from torchdrug import  layers, datasets,transforms,core
from torchdrug.core import Registry as R
from torchdrug.layers import geometry

import torch
from torchdrug import data

from torch_scatter import scatter_add
import torch.nn as nn
from torchdrug import utils
from torch.utils import checkpoint
import torch.nn.functional as F
import torch.optim as optim

import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据集获取

In [2]:
EnzymeCommission = R.search("datasets.EnzymeCommission")
PV = R.search("transforms.ProteinView")
trans = PV(view = "residue")
dataset = EnzymeCommission("~/scratch/protein-datasets/", test_cutoff=0.95, 
                           atom_feature="full", bond_feature="full", verbose=1, transform = trans)

# 只保留alpha碳的简化格式
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()], 
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)
                                                                 ],
                                                    edge_feature="gearnet"
                                                    )

15:36:18   Extracting /home/xiaotong/scratch/protein-datasets/EnzymeCommission.zip to /home/xiaotong/scratch/protein-datasets


Loading /home/xiaotong/scratch/protein-datasets/EnzymeCommission/enzyme_commission.pkl.gz: 100%|██████████| 18716/18716 [00:49<00:00, 377.14it/s]


In [3]:
graphs = dataset[56:58]
graphs = [element["graph"] for element in graphs]
graphs = data.Protein.pack(graphs)
print("\n\n")
graph = graph_construction_model(graphs)
print(graph)




PackedProtein(batch_size=2, num_atoms=[264, 308], num_bonds=[4992, 5924], num_residues=[264, 308])


In [4]:
print(graph.num_nodes)
print(graph.batch_size)
print(graph.edge_list)


tensor([264, 308])
2
tensor([[ 66,  68,   6],
        [ 57,  59,   6],
        [ 58,  60,   6],
        ...,
        [445, 493,   0],
        [480, 493,   0],
        [481, 493,   0]])


# 关系卷积神经网络，获取多个不同的嵌入

In [5]:
class relationalGraph(layers.MessagePassingBase):
    
    def __init__(self, input_dim, output_dim, num_relation, edge_input_dim=None, batch_norm=False, activation="relu"):
        super(relationalGraph, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.num_relation = num_relation
        self.edge_input_dim = edge_input_dim

        if batch_norm:
            self.batch_norm = nn.BatchNorm1d(output_dim)
        else:
            self.batch_norm = None
        if isinstance(activation, str):
            self.activation = getattr(F, activation)
        else:
            self.activation = activation

        self.self_loop = nn.Linear(input_dim, output_dim)
        self.linear = nn.Linear(input_dim, output_dim)
        if edge_input_dim:
            self.edge_linear = nn.Linear(edge_input_dim, input_dim)
        else:
            self.edge_linear = None
            
    def trans(self, A, graph):
    
        Degree_inv_sqrt = torch.diag(torch.pow(torch.sum(A, dim=1), -0.5))
        A_norm = torch.mm(torch.mm(Degree_inv_sqrt, A), Degree_inv_sqrt)
        
        n_rel = graph.num_relation
        n = A_norm.size(0)
        n_rel = n_rel.item()  # 将 n_rel 从 Tensor 转换为 int
        assert n % n_rel == 0, "n must be divisible by n_rel"
        
        block_size = n // n_rel
        
        # 初始化一个张量来存储累加结果
        accumulated = torch.zeros_like(A_norm[:block_size])
        
        # 将后面的所有块累加到第一块
        for i in range(n_rel):
            accumulated += A_norm[i * block_size: (i + 1) * block_size]
        
        # 用累加后的第一块替换原始矩阵的第一块
        A_trans = accumulated
    
        return A_trans

    def message_and_aggregate(self, graph, input, new_edge_list):
        assert graph.num_relation == self.num_relation
        device = input.device  # Ensure device consistency
        
        if new_edge_list is None:
            node_in, node_out, relation = graph.edge_list.t().to(device)
            node_out = node_out * self.num_relation + relation
        
            degree_out = scatter_add(graph.edge_weight, node_out, dim_size=graph.num_node * graph.num_relation)
            edge_weight = graph.edge_weight / degree_out[node_out]
            adjacency = utils.sparse_coo_tensor(torch.stack([node_in, node_out]), edge_weight,
                                                (graph.num_node, graph.num_node * graph.num_relation))
            update = torch.sparse.mm(adjacency.t().to(device), input.to(device))
        else:
            adjacency = self.trans(new_edge_list, graph).to(device)
            update = torch.mm(adjacency.t().to(device), input.to(device))
        
        if self.edge_linear:
            edge_input = graph.edge_feature.float().to(device)
            edge_input = self.edge_linear(edge_input)
            edge_weight = edge_weight.unsqueeze(-1).to(device)
            edge_update = scatter_add(edge_input * edge_weight, node_out, dim=0,
                                      dim_size=graph.num_node * graph.num_relation)
            update += edge_update

        return update

    def combine(self, input, update):
        # 自环特征
        device = input.device
        self.linear.to(device)  # Ensure the linear layers are on the correct device
        self.self_loop.to(device)
        input = input.repeat(self.num_relation, 1).to(device)
        loop_update = self.self_loop(input).to(device)
        
        output = self.linear(update)+loop_update
        if self.batch_norm:
            output = self.batch_norm(output)
        if self.activation:
            output = self.activation(output)
        return output
    
    def forward(self, graph, input, new_edge_list=None):
        
        if self.gradient_checkpoint:
            update = checkpoint.checkpoint(self._message_and_aggregate, *graph.to_tensors(), input, new_edge_list)
        else:
            update = self.message_and_aggregate(graph, input, new_edge_list)
        output = self.combine(input, update).view(graph.num_relation, input.size(0), -1)
        return output

#### 测试

In [6]:
input_dim = graph.node_feature.shape[-1]
output_dim = 128
num_relations = graph.num_relation

input = graph.node_feature.float()
model = relationalGraph(input_dim, output_dim, num_relations)
relational_output = model(graph, input, new_edge_list = None)
print("output: ", relational_output.shape)
print("output: ", relational_output.grad_fn)



output:  torch.Size([7, 572, 128])
output:  <ViewBackward object at 0x7f207ba18d00>


# 重连接模块

### window self attention + gumble softmax

In [23]:
class Rewirescorelayer(nn.Module):
    def __init__(self, in_features, out_features, num_heads, window_size, k, temperature=0.5, dropout=0.1):
        super(Rewirescorelayer, self).__init__()
        self.num_heads = num_heads
        self.out_features = out_features
        self.window_size = window_size
        self.temperature = temperature
        self.dropout = nn.Dropout(dropout)
        self.k = k
        
        self.query = nn.Linear(in_features, out_features * num_heads)
        self.key = nn.Linear(in_features, out_features * num_heads)
        self.scale = 1 / (out_features ** 0.5)

    def split_windows(self, tensor, index, window_size, device):
        result = []
        index_list = []
        start = 0

        for idx in index:
            end = start + idx - 1
            while start <= end:
                if start + window_size <= end:
                    result.append(tensor[:, start:start + window_size, :])
                    index_list.append([start, start + window_size])
                    start += window_size
                else:
                    padding_rows = window_size - (end - start + 1)
                    restart = start - padding_rows
                    result.append(tensor[:, restart:restart + window_size, :])
                    index_list.append([restart, restart + window_size])

                    start = end+1
        
        result_tensor = torch.stack(result, dim=1).to(device)
        return result_tensor, index_list

    def gumbel_softmax_top_k(self, logits, tau=1.0, hard=False):
        gumbels = -torch.empty_like(logits).exponential_().log()
        gumbels = (logits + gumbels) / tau

        y_soft = F.softmax(gumbels, dim=-1)

        if hard:
            topk_indices = logits.topk(self.k, dim=-1)[1]
            y_hard = torch.zeros_like(logits).scatter_(-1, topk_indices, 1.0)
            y = (y_hard - y_soft).detach() + y_soft
        else:
            y = y_soft

        return y

    def windows2adjacent(self, windows, index_list, output, device):
        # 确保所有张量在相同设备上
        output = output.to(device)
        windows = windows.to(device)

        # 创建一个新的张量来存储更新后的输出
        new_output = torch.zeros_like(output, device=device)

        # 填充新输出张量
        for i, index in enumerate(index_list):
            start, end = index
            new_output[:, start:end, start:end] = torch.clamp(new_output[:, start:end, start:end] + windows[:, i, :, :], 0, 1)

        # 获取输出张量的形状
        num_relations, num_nodes, _ = new_output.shape
        
        # 创建一个新的结果张量
        result = torch.zeros(num_relations * num_nodes, num_relations * num_nodes, device=device)

        # 填充结果张量
        for i in range(num_relations):
            result[i * num_nodes:(i + 1) * num_nodes, i * num_nodes:(i + 1) * num_nodes] = result[i * num_nodes:(i + 1) * num_nodes, i * num_nodes:(i + 1) * num_nodes] + new_output[i]

        return result

    def forward(self, graph, node_features):
        device = node_features.device
        num_relations = node_features.size(0)
        num_nodes = node_features.size(1)
        index = graph.num_nodes.tolist()
        
        Q = self.query(node_features).view(num_relations, num_nodes, self.num_heads, self.out_features).permute(0, 2, 1, 3)
        K = self.key(node_features).view(num_relations, num_nodes, self.num_heads, self.out_features).permute(0, 2, 1, 3)
        Q = Q.reshape(num_relations * self.num_heads, num_nodes, self.out_features).to(device)
        K = K.reshape(num_relations * self.num_heads, num_nodes, self.out_features).to(device)
        
        
        output = torch.zeros(num_relations, num_nodes, num_nodes).to(device)
        result = torch.zeros(num_relations * num_nodes, num_relations * num_nodes).to(device)
       
        Q_windows, Q_index = self.split_windows(Q, index, self.window_size, device)
        print(Q_index)
        K_windows, _ = self.split_windows(K, index, self.window_size, device)
        
        scores = torch.einsum('b h i e, b h j e -> b h i j', Q_windows, K_windows) / self.scale                                 # (num_relations*num_heads, num_windows, window_size, window_size)
        attn = scores.softmax(dim=-1).view(num_relations, self.num_heads, -1, self.window_size, self.window_size).mean(dim=1)   # (num_relations, num_windows, window_size, window_size)
        attn = self.gumbel_softmax_top_k(attn, tau=self.temperature, hard=True)                                                 # (num_relations, num_windows, window_size, window_size)
        
        result = result + self.windows2adjacent(attn, Q_index, output, device)
        
        return result


#### 测试

In [24]:
input_dim = relational_output.shape[-1]
output_dim = 256
num_heads = 8
window_size = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k = 3
num_nodes = relational_output.size(0)

start = time.time()
relational_output = relational_output.to(device)
module = Rewirescorelayer(input_dim, output_dim, num_heads, window_size, k).to(device)
attn_output = module(graph, relational_output)
end = time.time()
print(f"运行时间: {end - start:.6f} 秒")
print(attn_output.shape)

print(attn_output)
indices = torch.nonzero(attn_output, as_tuple=True)
print(indices)


[[0, 10], [10, 20], [20, 30], [30, 40], [40, 50], [50, 60], [60, 70], [70, 80], [80, 90], [90, 100], [100, 110], [110, 120], [120, 130], [130, 140], [140, 150], [150, 160], [160, 170], [170, 180], [180, 190], [190, 200], [200, 210], [210, 220], [220, 230], [230, 240], [240, 250], [250, 260], [254, 264], [264, 274], [274, 284], [284, 294], [294, 304], [304, 314], [314, 324], [324, 334], [334, 344], [344, 354], [354, 364], [364, 374], [374, 384], [384, 394], [394, 404], [404, 414], [414, 424], [424, 434], [434, 444], [444, 454], [454, 464], [464, 474], [474, 484], [484, 494], [494, 504], [504, 514], [514, 524], [524, 534], [534, 544], [544, 554], [554, 564], [562, 572]]
运行时间: 0.054884 秒
torch.Size([4004, 4004])
tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], device='cuda:0',
       gra

### 测试不同degree进行采样

In [25]:
import torch
import torch.nn.functional as F

def gumbel_softmax_sample(logits, tau):
    # 从Gumbel(0, 1)分布中采样
    gumbel_noise = -torch.log(-torch.log(torch.rand_like(logits)))
    y = logits + gumbel_noise
    return F.softmax(y / tau, dim=-1)

def bernoulli_sampling_with_different_thresholds(probs, thresholds, tau=1.0):
    # 对数概率
    logits = torch.log(probs) - torch.log(1 - probs)
    # 进行Gumbel-Softmax采样
    y = gumbel_softmax_sample(logits, tau)
    # 硬化处理，根据每行的不同阈值
    z = (y > thresholds.unsqueeze(1)).float()
    return z

# 示例矩阵
n = 10
P = torch.rand(n, n)

# 为每一行设置不同的阈值
thresholds = torch.tensor([0.2, 0.5, 0.7, 0.9, 0.1, 0.3, 0.6, 0.8, 0.4, 0.2])

# 进行可微分伯努利采样并硬化处理
tau = 0.1  # 温度参数
sampled_matrix = bernoulli_sampling_with_different_thresholds(P, thresholds, tau)

#print("Probability Matrix:\n", P)
print("Thresholds:\n", thresholds)
print("Sampled Matrix:\n", sampled_matrix)


Thresholds:
 tensor([0.2000, 0.5000, 0.7000, 0.9000, 0.1000, 0.3000, 0.6000, 0.8000, 0.4000,
        0.2000])
Sampled Matrix:
 tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])


# Diffusion模块

### Rewired_gearnet 用于diffusion模块

In [26]:
class RewireGearnet(nn.Module):
    gradient_checkpoint = False

    def __init__(self, input_dim, output_dim, num_relation, edge_input_dim=None, batch_norm=False, activation="relu"):
        super(RewireGearnet, self).__init__()
        self.num_relation = num_relation
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.linear = nn.Linear(num_relation * input_dim, output_dim)
        self.self_loop = nn.Linear(input_dim, output_dim)
        self.batch_norm = nn.BatchNorm1d(output_dim) if batch_norm else None
        self.activation = getattr(F, activation) if activation else None
        self.edge_linear = nn.Linear(edge_input_dim, output_dim) if edge_input_dim else None

    def trans(self, A, graph):
        n_rel = graph.num_relation
        n = A.size(0)
        n_rel = n_rel.item()  # 将 n_rel 从 Tensor 转换为 int
        assert n % n_rel == 0, "n must be divisible by n_rel"
        
        block_size = n // n_rel
        
        # 初始化一个张量来存储累加结果
        accumulated = torch.zeros_like(A[:block_size])
        
        # 将后面的所有块累加到第一块
        for i in range(n_rel):
            accumulated += A[i * block_size: (i + 1) * block_size]
        
        # 用累加后的第一块替换原始矩阵的第一块
        A_trans = accumulated
    
        return A_trans

    def message_and_aggregate(self, graph, input, new_edge_list):
        assert graph.num_relation == self.num_relation

        device = input.device  # Ensure device consistency

        if new_edge_list is None:
            node_in, node_out, relation = graph.edge_list.t().to(device)
            node_out = node_out * self.num_relation + relation
            adjacency = torch.sparse_coo_tensor(
                torch.stack([node_in, node_out]),
                graph.edge_weight.to(device),
                (graph.num_node, graph.num_node * graph.num_relation),
                device=device
            )
            update = torch.sparse.mm(adjacency.t(), input)
        else:
            adjacency = self.trans(new_edge_list, graph).to(device)
            update = torch.mm(adjacency.t(), input.to(device))
        
        if self.edge_linear:
            edge_input = graph.edge_feature.float().to(device)
            edge_input = self.edge_linear(edge_input)
            edge_weight = graph.edge_weight.unsqueeze(-1).to(device)
            edge_update = scatter_add(
                edge_input * edge_weight, node_out, dim=0,
                dim_size=graph.num_node * graph.num_relation
            )
            update += edge_update
            
        return update.view(graph.num_node, self.num_relation * self.input_dim).to(device)

    def combine(self, input, update):
        device = input.device
        self.linear.to(device)  # Ensure the linear layers are on the correct device
        self.self_loop.to(device)
        if self.batch_norm:
            self.batch_norm.to(device)
        
        output = self.linear(update) + self.self_loop(input)
        if self.batch_norm:
            output = self.batch_norm(output)
        if self.activation:
            output = self.activation(output)
        return output

    def forward(self, graph, input, new_edge_list=None):
        """
        Perform message passing over the graph(s).

        Parameters:
            graph (Graph): graph(s)
            input (Tensor): node representations of shape :math:`(|V|, ...)`
        """
        if self.gradient_checkpoint:
            update = checkpoint.checkpoint(self.message_and_aggregate, graph, input)
        else:
            update = self.message_and_aggregate(graph, input, new_edge_list)
        output = self.combine(input, update)
        return output

#### 测试

In [27]:
input_dim = graph.node_feature.shape[-1]
output_dim = 512
num_relations = graph.num_relation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


new_node_feature = RewireGearnet(input_dim, output_dim, num_relations)(graph, graph.node_feature.to(device).float(), attn_output).to(device)

print(new_node_feature.shape)
print(new_node_feature)

torch.Size([572, 512])
tensor([[0.1539, 0.2224, 0.0000,  ..., 0.0000, 0.4023, 0.0000],
        [0.4200, 0.3484, 0.0000,  ..., 0.3663, 0.2991, 0.0000],
        [0.3223, 0.2434, 0.0000,  ..., 0.1596, 0.1579, 0.0000],
        ...,
        [0.3084, 0.1414, 0.0000,  ..., 0.0000, 0.1376, 0.0000],
        [0.2833, 0.1915, 0.1157,  ..., 0.0000, 0.4115, 0.0000],
        [0.1712, 0.2559, 0.0000,  ..., 0.1079, 0.1834, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)


# 最终模型

In [28]:
class DGMGearnet(nn.Module, core.Configurable):

    def __init__(self, input_dim, hidden_dims, score_dim, num_relation, num_heads, window_size, k, edge_input_dim=None, num_angle_bin=None,
                 short_cut=False, batch_norm=False, activation="relu", concat_hidden=True, readout="sum"):
        super(DGMGearnet, self).__init__()

        #if not isinstance(hidden_dims, Sequence):
            #hidden_dims = [hidden_dims]
        self.input_dim = input_dim
        self.output_dim = sum(hidden_dims) if concat_hidden else hidden_dims[-1]
        self.dims = [input_dim] + list(hidden_dims)
        self.score_dim = score_dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.k = k
        self.edge_dims = [edge_input_dim] + self.dims[:-1]
        self.num_relation = num_relation
        self.num_angle_bin = num_angle_bin
        self.short_cut = short_cut
        self.concat_hidden = concat_hidden
        self.batch_norm = batch_norm

        self.layers = nn.ModuleList()
        self.score_layers = nn.ModuleList()
        for i in range(len(self.dims) - 1):
            
            self.score_layers.append(relationalGraph(self.dims[i], self.score_dim, num_relation, 
                                                     edge_input_dim=None, batch_norm=False, activation="relu")) 

            
            self.score_layers.append(Rewirescorelayer(self.score_dim, self.dims[i+1], self.num_heads, self.window_size, 
                                            self.k, temperature=0.5, dropout=0.1))
            
            self.layers.append(RewireGearnet(self.dims[i], self.dims[i + 1], num_relation,
                                            edge_input_dim=None, batch_norm=False, activation="relu"))
        
        if num_angle_bin:
            self.spatial_line_graph = layers.SpatialLineGraph(num_angle_bin)
            self.edge_layers = nn.ModuleList()
            for i in range(len(self.edge_dims) - 1):
                self.edge_layers.append(layers.GeometricRelationalGraphConv(
                    self.edge_dims[i], self.edge_dims[i + 1], num_angle_bin, None, batch_norm, activation))

        if batch_norm:
            self.batch_norms = nn.ModuleList()
            for i in range(len(self.dims) - 1):
                self.batch_norms.append(nn.BatchNorm1d(self.dims[i + 1]))

        if readout == "sum":
            self.readout = layers.SumReadout()
        elif readout == "mean":
            self.readout = layers.MeanReadout()
        else:
            raise ValueError("Unknown readout `%s`" % readout)

    def forward(self, graph, input, edge_list=None, all_loss=None, metric=None):
        """
        Compute the node representations and the graph representation(s).

        Parameters:
            graph (Graph): :math:`n` graph(s)
            input (Tensor): input node representations
            all_loss (Tensor, optional): if specified, add loss to this tensor
            metric (dict, optional): if specified, output metrics to this dict

        Returns:
            dict with ``node_feature`` and ``graph_feature`` fields:
                node representations of shape :math:`(|V|, d)`, graph representations of shape :math:`(n, d)`
        """
        hiddens = []
        layer_input = input
        if self.num_angle_bin:
            line_graph = self.spatial_line_graph(graph)
            edge_input = line_graph.node_feature.float()

        for i in range(len(self.layers)):
            
            
            relational_output = self.score_layers[2*i](graph, layer_input, edge_list)
            new_edge_list = self.score_layers[2*i+1](graph, relational_output)
            
            hidden = self.layers[i](graph, layer_input, new_edge_list)
            
            if self.short_cut and hidden.shape == layer_input.shape:
                hidden = hidden + layer_input
            
            if self.num_angle_bin:
                edge_hidden = self.edge_layers[i](line_graph, edge_input)
                edge_weight = graph.edge_weight.unsqueeze(-1)
                if new_edge_list is None:
                    node_out = graph.edge_list[:, 1] * self.num_relation + graph.edge_list[:, 2]
                else:
                    node_out = new_edge_list[:, 1] * self.num_relation + new_edge_list[:, 2]
                
                    update = scatter_add(edge_hidden * edge_weight, node_out, dim=0,
                                        dim_size=graph.num_node * self.num_relation)
                update = update.view(graph.num_node, self.num_relation * edge_hidden.shape[1])
                update = self.layers[i].linear(update)
                update = self.layers[i].activation(update)
                hidden = hidden + update
                edge_input = edge_hidden
                
            if self.batch_norm:
                hidden = self.batch_norms[i](hidden)
                
            hiddens.append(hidden)
            layer_input = hidden
            edge_list = new_edge_list

        if self.concat_hidden:
            node_feature = torch.cat(hiddens, dim=-1)
        else:
            node_feature = hiddens[-1]
        graph_feature = self.readout(graph, node_feature)

        return {
            "graph_feature": graph_feature,
            "node_feature": node_feature
        }

### 测试

In [44]:
input_dim = graph.node_feature.shape[-1]
hidden_dims = [512, 512, 512, 512, 512]
score_dim = 128
num_relations = graph.num_relation
num_heads = 8
window_size = 10
k = 5


output = DGMGearnet(input_dim, hidden_dims, score_dim, num_relations, num_heads, window_size, k).to(device)(graph.to(device), graph.node_feature.to(device).float())

In [45]:
print(output["node_feature"])
print(output["node_feature"].shape)
print("\n")

print(output["graph_feature"])
print(output["graph_feature"].shape)

tensor([[ 0.0000,  0.0210,  0.0000,  ...,  1.7995,  3.3497,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  3.9993,  4.2867,  0.2628],
        [ 0.3706,  0.0000,  0.0000,  ...,  0.0000,  6.6846,  0.2074],
        ...,
        [ 0.1841,  0.0000,  0.0000,  ...,  0.0000,  0.8430,  8.7589],
        [ 0.2833,  0.0000,  0.0000,  ...,  0.0000,  0.7668, 12.4685],
        [ 0.6847,  0.0000,  0.0310,  ...,  0.0000,  3.2075,  0.0000]],
       device='cuda:0', grad_fn=<CatBackward>)
torch.Size([572, 2560])


tensor([[ 117.4508,    5.4980,    8.1354,  ...,  868.7866, 1522.7261,
         1172.5823],
        [ 138.2602,    5.0186,   11.3572,  ..., 1045.4672, 1846.1328,
         1237.1732]], device='cuda:0', grad_fn=<ScatterAddBackward>)
torch.Size([2, 2560])


### 验证反向传播

In [20]:
model = DGMGearnet(input_dim, hidden_dims, score_dim, num_relations, num_heads, window_size, k).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

output = model(graph.to(device), graph.node_feature.to(device).float())

output = output["graph_feature"]

# 定义一个简单的损失函数，例如均方误差
criterion = F.binary_cross_entropy_with_logits
target = torch.rand_like(output)

# 计算损失
loss = criterion(output, target, reduction="mean")
print(loss)
# 反向传播
loss.backward()

# 检查梯度是否为 NaN 或 inf
for name, param in model.named_parameters():
    if param.grad is not None:
        if torch.isnan(param.grad).any():
            print(f"Gradient of {name} contains NaN values.")
        if torch.isinf(param.grad).any():
            print(f"Gradient of {name} contains inf values.")
        else:
            print(f"Layer {name} - Gradient_norm: {torch.norm(param.grad)}")
            
    else:
        print(f"No gradient found for {name}")

# 检查损失是否为 NaN
if torch.isnan(loss):
    print("Loss is NaN.")
        
# 更新参数
optimizer.step()

tensor(406.9637, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Layer layers.0.linear.weight - Gradient_norm: 406.1522216796875
Layer layers.0.linear.bias - Gradient_norm: 115.85568237304688
Layer layers.0.self_loop.weight - Gradient_norm: 30.428178787231445
Layer layers.0.self_loop.bias - Gradient_norm: 115.85568237304688
Layer layers.1.linear.weight - Gradient_norm: 3432.73779296875
Layer layers.1.linear.bias - Gradient_norm: 60.82954025268555
Layer layers.1.self_loop.weight - Gradient_norm: 254.52467346191406
Layer layers.1.self_loop.bias - Gradient_norm: 60.82954025268555
Layer layers.2.linear.weight - Gradient_norm: 3126.965087890625
Layer layers.2.linear.bias - Gradient_norm: 25.393918991088867
Layer layers.2.self_loop.weight - Gradient_norm: 231.603515625
Layer layers.2.self_loop.bias - Gradient_norm: 25.393918991088867
Layer layers.3.linear.weight - Gradient_norm: 2956.97314453125
Layer layers.3.linear.bias - Gradient_norm: 10.630610466003418
Layer laye